In [ ]:
from syft import Worker
import syft as sy
worker = Worker.named("test-domain-1", processes=1, reset=False)
root_domain_client = worker.root_client

In [ ]:
# !rm /var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite

# Download mock and submit project

## Get mock

In [ ]:
guest_domain_client = worker.guest_client
guest_client = guest_domain_client.login(email="jane@caltech.edu", password="abc123")

In [ ]:
ds = guest_domain_client.datasets[0]

In [ ]:
asset = ds.assets["bikes"]

In [ ]:
fixed_df = asset.mock

## Selecting a column

When you read a CSV, you get a kind of object called a DataFrame, which is made up of rows and columns. You get columns out of a DataFrame the same way you get elements out of a dictionary.

Here's an example:

In [ ]:
fixed_df['Berri 1']

## Plotting a column

Just add .plot() to the end! How could it be easier? =)

We can see that, unsurprisingly, not many people are biking in January, February, and March,

In [ ]:
fixed_df['Berri 1'].plot()

We can also plot all the columns just as easily. We'll make it a little bigger, too. You can see that it's more squished together, but all the bike paths behave basically the same -- if it's a bad day for cyclists, it's a bad day everywhere.

In [ ]:
fixed_df.plot(figsize=(15, 10))

## Putting that all together

Here's the code we needed to write do draw that graph, all together:



In [ ]:
res = df['Berri 1'].plot()

In [ ]:
@sy.syft_function(input_policy=sy.ExactMatch(df=asset.mock),
                  output_policy=sy.SingleExecutionExactOutput())
def sum_trade_value_mil(df):
    return df['Berri 1']

Request code execution

In [ ]:
req = guest_domain_client.api.services.code.request_code_execution(sum_trade_value_mil)


In [ ]:
submitted_code = guest_domain_client.code[0]

In [ ]:
assert guest_domain_client.api.services.code.get_all()

Create and submit project

In [ ]:
new_project = sy.Project(name="My pandas project 2",
                         description="Hi, I would like to plot the Berri 1 column.")

In [ ]:
new_project.add_request(obj=submitted_code, permission=sy.UserCodeStatus.EXECUTE)

In [ ]:
guest_domain_client.submit_project(new_project)